In [1]:
import json
import elasticsearch
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
import os
os.getcwd()

'd:\\Projects\\AI-Restaurent-Chat-bot\\notebooks'

In [3]:
# change path for base folder
base_folder = 'D:/Projects/AI-Restaurent-Chat-bot/'
input_data_folder = base_folder+'input_data/'

In [4]:
# openai api
from openai import OpenAI
client = OpenAI()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
import pandas as pd
import json

# Read the Excel file
df = pd.read_csv(input_data_folder+"cakes_data.csv")

# Convert the DataFrame to a list of dictionaries (JSON-like structure)
data = df.to_dict(orient='records')

# Save to a JSON file for indexing
with open("menu_data.json", "w") as f:
    json.dump(data, f, indent=4)


In [9]:
df.head()

,id,name,minutes,tags,n_steps,steps,ingredients,n_ingredients,user_id,rating,...,rating_category,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),calorie_status,price
0,142,almond fudge banana cake,110,"['weeknight', 'time-to-make', 'course', 'prepa...",13,"['mash bananas and set aside', 'beat sugar and...","['dole banana', 'sugar', 'margarine', 'eggs', ...",11,914114,4,...,tasty,224.8,14,87,10,7,9,11,Low Calory,24
1,520,apple pie cake,40,"['60-minutes-or-less', 'time-to-make', 'course...",4,"['cream butter , sugar and the egg', 'mix toge...","['sugar', 'egg', 'salt', 'butter', 'flour', 'b...",12,22504,3,...,so so,359.1,26,136,18,7,40,16,Low Calory,20
2,822,heloise s cake mix cookies,8,"['15-minutes-or-less', 'time-to-make', 'course...",4,"['mix all ingredients well to remove lumps', '...","['cake mix', 'vegetable oil', 'eggs']",3,36327,5,...,tasty,137.4,11,46,6,2,5,5,Low Calory,44
3,916,momma s fair funnel cake,20,"['30-minutes-or-less', 'time-to-make', 'course...",9,['beat eggs and sugar together and then add th...,"['eggs', 'sugar', 'milk', 'flour', 'salt', 'ba...",7,13640,5,...,tasty,398.9,9,34,16,27,13,23,Low Calory,31
4,1633,mc donald s pancakes,20,"['30-minutes-or-less', 'time-to-make', 'course...",4,"['put all ingredients into a blender', 'blend ...","['carbonated lemon-lime beverage', 'egg', 'sug...",5,60908,5,...,tasty,225.6,14,39,16,8,10,10,Low Calory,31


In [8]:
from elasticsearch import Elasticsearch, helpers

# Connect to Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Load the JSON data
with open("menu_data.json", "r") as f:
    menu_data = json.load(f)

# Index name
index_name = "menu_index"

# Index the data
def index_data():
    actions = [
        {
            "_index": index_name,
            "_id": i,
            "_source": data
        }
        for i, data in enumerate(menu_data)
    ]
    
    # Bulk index the data
    helpers.bulk(es, actions)

# Run the indexing function
index_data()


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x00000169C1DA2ED0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it))

In [87]:
def elastic_search(query, dish_name=None):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["Food items^3", "ingredients^3", "price", "fat perc", "calories perce"],
                            "type": "best_fields"
                        }
                    }
                ]
            }
        }
    }

    # Add a filter by dish_name if it's provided
    if dish_name:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "Food items.keyword": dish_name
            }
        }

    # Perform the search
    response = es.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        doc = hit['_source']
        result_docs.append(doc)
        
    return result_docs


[{'id': '142_2',
  'question': 'How many calories does it have?',
  'section': 'calories',
  'text': 'The almond fudge banana cake has 224.8 calories.',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_3',
  'question': 'How much total fat does it contain?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 14 grams of total fat (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_4',
  'question': 'What is the sugar content?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 87 grams of sugar (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_5',
  'question': 'How much sodium is in this dish?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 10 grams of sodium (PDV).',
  'dish_name': 'almond fudge banana cake'}]

In [43]:
# openai api
from openai import OpenAI
client = OpenAI()
openai_api_key = os.getenv("OPENAI_API_KEY")


def generate_response_from_llm(search_results):
    prompt = "You are a helpful assistant. Answer the following question based on the information provided:\n\n"

    for result in search_results:
        prompt += f"Food item: {result['Food items']}\n"
        prompt += f"Ingredients: {result['ingredients']}\n"
        prompt += f"Price: {result['price']}\n"
        prompt += f"Calories: {result['calories perce']}%\n\n"

    prompt += "What can you tell me about the ingredients of these dishes?"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].text.strip()

# Example usage
query = "What are the ingredients in banana pudding cake?"
search_results = elastic_search(query)
llm_response = generate_response_from_llm(search_results)

print(llm_response)

ObjectApiResponse({'name': '81d6d96573fa', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Knoc8xY-Teq-lYkTwu4Eww', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [44]:

index_name = "menu-items"

# Check if index exists
if not es.indices.exists(index=index_name):
    # Create the index with the correct mapping
    response = es.indices.create(
        index=index_name,
        body={
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "category": {"type": "text"},
                    "dishes": {
                        "type": "nested",
                        "properties": {
                            "dish name": {"type": "text"},
                            "documents": {
                                "type": "nested",
                                "properties": {
                                    "id": {"type": "keyword"},
                                    "question": {"type": "text"},
                                    "section": {"type": "text"},
                                    "text": {"type": "text"}
                                }
                            }
                        }
                    }
                }
            }
        }
    )
    print("Index creation response:", response)
else:
    print(f"Index '{index_name}' already exists.")


Index 'menu-items' already exists.
